In [3]:
import odoolib
import pandas as pd
import numpy as np

In [10]:
class OddoConn:
    def __init__(self,conn_params):
        self.listas_recurrentes = {
            'MAESTRA':['x_studio_sku_unidad_de_negocio','x_name','x_studio_stage_id','x_studio_variable_de_marcado','x_studio_candidato_a_analisis_fisico']
        }
        self.conn_params = conn_params
        self.connect()
        self.resultadoBusqueda = None
    
    def connect(self):
        self.conexion = odoolib.get_connection(
                hostname=self.conn_params['ODOO_HOSTNAME'],
                database=self.conn_params['ODOO_DATABASE'],
                login=self.conn_params['ODOO_USERNAME'],
                password=self.conn_params['ODOO_PASSWORD'],
                port=443,
                protocol='jsonrpcs')
        
    def limpiar_resultados(self,x):
        x['x_studio_stage_id'] = x['x_studio_stage_id'][1]
        return x
    
    def getDataFromModel(self,modelo,lista_filtros,lista_campos,header=None):

        # SET HEADER
        if not header:  header = ['id'] + lista_campos
        else:           header = ['id'] + header

        # OBTENER DATOS
        res = self.conexion.get_model(modelo)
        res = res.search_read(lista_filtros,lista_campos,limit=5)
        res = list( map(self.limpiar_resultados,res) )

        # CREAR DATAFRANE
        res = list( map(lambda x:list(x.values()),res) )
        res = np.array(res)
        res = pd.DataFrame(data=res,columns=header)

        return res
    
    def downloadExcel(self):
        pass

    def maestra(self,empresa):
        modelo = 'x_productos'
        filtros = [('x_studio_unidades_de_negocio','=',empresa)]
        campos = ['x_studio_sku_unidad_de_negocio','x_name','x_studio_stage_id','x_studio_variable_de_marcado','x_studio_candidato_a_analisis_fisico']
        header = ['SKU unidad negocio','SKU','Etapa','EVA','Analisis fisivo']
        return self.getDataFromModel(modelo,filtros,campos,header)



In [11]:
conn_params = {
    'ODOO_USERNAME' : 'acceso@asalvo.cl',
    'ODOO_PASSWORD' : '1234',
    'ODOO_HOSTNAME' : 'sinergia-lab-cencorep-ambiente-test-9601418.dev.odoo.com',
    'ODOO_DATABASE' : 'sinergia-lab-cencorep-ambiente-test-9601418'
}
odoo = OddoConn(conn_params)
odoo.maestra)

,id,SKU unidad negocio,SKU,Etapa,EVA,Analisis fisivo
0,183729,1882498,1882498SMK,Completado,True,False
1,185396,1559896,1559896SMK,Completado,True,False
2,186151,1882552,1882552SMK,Completado,True,False
3,186152,1882553,1882553SMK,Completado,True,False
4,186277,1869415,1869415SMK,Completado,True,False


In [145]:
# ================
# CREAR DATAFRAME
# ================

header = ['ID','SKU unidad negocio','SKU','Etapa','eva','Analisis fisico']
df_resultados = list( map(lambda x:list(x.values()),resultadoBusqueda) )
df_resultados = np.array(df_resultados)
df_resultados = pd.DataFrame(data=df_resultados,columns=header)
df_resultados = df_resultados[['SKU unidad negocio','SKU','ID','Etapa','eva','Analisis fisico']]
df_resultados

,SKU unidad negocio,SKU,ID,Etapa,eva,Analisis fisico
0,1882498,1882498SMK,183729,Completado,True,False
1,1559896,1559896SMK,185396,Completado,True,False
2,1882552,1882552SMK,186151,Completado,True,False
3,1882553,1882553SMK,186152,Completado,True,False
4,1869415,1869415SMK,186277,Completado,True,False
...,...,...,...,...,...,...
66557,1970661,1970661SMK,484796,No completado - Nuevo,False,False
66558,1970509,1970509SMK,484797,No completado - Nuevo,False,False
66559,10000097589,10000097589SMK,484888,No completado - Nuevo,False,False
66560,10000093009,10000093009SMK,484889,No completado - Nuevo,False,False


In [146]:
# ==========
# GUARDAR
# ==========

# df_resultados.to_csv('Productos.csv')
df_resultados.to_excel('Productos.xlsx')